In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from xgboost import plot_importance
from imblearn.under_sampling import RandomUnderSampler
import sklearn as sk
import pickle

In [2]:
# Create a custom function for data transformation
# May as well import all of these features into the initial dataframe, you can cut it down..
# ..when you do the X,y train/test split
features = ['datetime',
            'latitude',
            'longitude',
            'absoluteAltitude',
            'magneticHeading',
            'trueHeading',
            'groundSpeed',
            'trueAirspeed',
            'pitch',
            'roll',
            'yaw',
            'angleOfAttack',
            'flightPathAngle',
            'verticalFlightPathAngle',
            'horizontalFlightPathAngle',
            'rollAcceleration',
            'pitchAcceleration',
            'yawAcceleration',
            'e1N1ng',
            'e2N1ng',
            'e1N2nf',
            'e2N2nf',
            'mainRotorRpm',
            'tailRotorRpm',
            'mainRotorTorque',
            'e1Itt',
            'e2Itt',
            'mainRotorInducedInflow',
            'flightDirectorPitch',
            'flightDirectorRoll',
            'climbOrDescentRate',
            'hasWeightOnWheels',
            'turnRate to degPerSec',
            'cyclicPositionPitch',
            'cyclicPositionRoll',
            'collectivePosition',
            'antiTorquePedalPosition',
            'throttlePosition',
            'e1Torque',
            'e2Torque'
           ]

In [3]:
def df_transformation(df):
    df_temp = df[features]
    df_temp['datetime'] = df_temp.datetime.dt.strftime('%Y-%m-%d %H:%M:%S').astype('datetime64[ns]')

    output = df_temp.groupby(by=["datetime"], dropna=False).mean().reset_index()
    
    return output 

In [4]:
# reading files from 9/19 dataset
df1 = pd.read_csv('SimData_2022.09.19_16.00.36.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df1 = df_transformation(df1)


df2 = pd.read_csv('SimData_2022.09.19_16.38.20.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df2 = df_transformation(df2)


df3 = pd.read_csv('SimData_2022.09.19_17.51.37.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df3 = df_transformation(df3)



# reading files from 9/20 dataset
df4 = pd.read_csv('SimData_2022.09.20_16.22.07.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df4 = df_transformation(df4)


df5 = pd.read_csv('SimData_2022.09.20_16.28.31.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df5 = df_transformation(df5)


# reading file from 9/22 dataset
df6 = pd.read_csv('SimData_2022.09.22_17.11.56.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df6 = df_transformation(df6)


# reading file from 10/06 dataset
df7 = pd.read_csv('SimData_2022.10.06_09.47.22.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)
df7 = df_transformation(df7)

C:\Users\kanev\AppData\Local\Temp\ipykernel_16020\17026926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['datetime'] = df_temp.datetime.dt.strftime('%Y-%m-%d %H:%M:%S').astype('datetime64[ns]')
C:\Users\kanev\AppData\Local\Temp\ipykernel_16020\17026926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['datetime'] = df_temp.datetime.dt.strftime('%Y-%m-%d %H:%M:%S').astype('datetime64[ns]')
C:\Users\kanev\AppData\Local\Temp\ipykernel_16020\17026926.py:3: SettingWithCopyWarning: 
A valu

In [5]:
#Combine files
df = pd.concat([df1, df2, df3, df4, df5, df6, df7]).reset_index()
len(df)

8487

In [6]:
df.head()

,index,datetime,latitude,longitude,absoluteAltitude,magneticHeading,trueHeading,groundSpeed,trueAirspeed,pitch,...,climbOrDescentRate,hasWeightOnWheels,turnRate to degPerSec,cyclicPositionPitch,cyclicPositionRoll,collectivePosition,antiTorquePedalPosition,throttlePosition,e1Torque,e2Torque
0,0,2022-09-19 20:01:09,39.358996,-74.434692,0.032500,343.437625,331.099250,0.137500,0.205666,0.219875,...,0.582125,1.0,-0.195750,24.912250,-4.404250,3.726,-38.213000,7.283500,-5.219500,-5.219500
1,1,2022-09-19 20:01:10,39.358996,-74.434692,0.054773,341.506909,329.168455,0.155500,0.170178,-0.247591,...,2.061705,1.0,-0.824295,0.949909,1.013386,3.726,-38.213000,39.997136,-3.727205,-3.727205
2,2,2022-09-19 20:01:11,39.358996,-74.434692,0.060000,341.011659,328.673114,0.018477,0.019431,-0.172818,...,1.114295,1.0,-0.669841,-0.176841,0.624068,3.726,-38.216068,87.756318,6.978659,6.978659
3,3,2022-09-19 20:01:12,39.358996,-74.434692,0.060000,341.334978,328.996422,0.005200,0.005306,-0.167422,...,0.181933,1.0,-0.309289,0.039356,0.124244,3.726,-38.228200,100.000000,28.398222,28.398222
4,4,2022-09-19 20:01:13,39.358996,-74.434692,0.060000,341.558604,329.220042,0.002542,0.002629,-0.172063,...,-0.000208,1.0,-0.118042,0.065479,0.052958,3.726,-38.282792,100.000000,56.248208,56.248208


# Surface Taxi - DataFrame Labelling

In [7]:
df_s = df.copy()
df_s['surface'] = 0

df_s.loc[(df_s['datetime'].between('2022-09-19 21:53:50' , '2022-09-19 21:55:55' , inclusive='both')),['surface']] = 1
df_s.loc[(df_s['datetime'].between('2022-09-19 22:04:00' , '2022-09-19 22:06:39' , inclusive='both')),['surface']] = 1
df_s.loc[(df_s['datetime'].between('2022-09-20 20:25:50' , '2022-09-20 20:27:54' , inclusive='both')),['surface']] = 1
df_s.loc[(df_s['datetime'].between('2022-09-22 21:16:37' , '2022-09-22 21:19:56' , inclusive='both')),['surface']] = 1
df_s.loc[(df_s['datetime'].between('2022-09-22 21:43:16' , '2022-09-22 21:43:41' , inclusive='both')),['surface']] = 1
df_s.loc[(df_s['datetime'].between('2022-10-06 13:52:16' , '2022-10-06 13:56:00' , inclusive='both')),['surface']] = 1

In [8]:
#orginal class imbalance
df_s['surface'].value_counts()

0    7625
1     862
Name: surface, dtype: int64

In [9]:
#call features used in model
model_features = ['hasWeightOnWheels',
                  'absoluteAltitude',
                  'groundSpeed',
                  'climbOrDescentRate',
                  'cyclicPositionPitch',
                  'cyclicPositionRoll',
                  'collectivePosition',
                  'antiTorquePedalPosition',
                  'throttlePosition',
                  'e1Torque',
                  'e2Torque'
                 ]

In [10]:
#call OverSampler using 'majority' 
rus = RandomUnderSampler(sampling_strategy='majority', random_state=777)

In [11]:
#set x and y to respective features
surface_x, surface_y = df_s[model_features], df_s['surface']

In [12]:
#resample
surface_x, surface_y = rus.fit_resample(surface_x, surface_y)

In [13]:
#new class balance
surface_y.value_counts()

0    862
1    862
Name: surface, dtype: int64

# Surface Taxi - XGBoost Training Model

In [14]:
#call XG Boost model and fit to surface data
surface_model = XGBClassifier(use_label_encoder=False, max_depth = 5, eta = .05, min_child_weight = 2)
surface_model.fit(surface_x, surface_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.05,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.0500000007, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=2,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [15]:
#export model for validation/testing
filename = 'final_surface_model.pkl'
pickle.dump(surface_model, open(filename, 'wb'))

In [16]:
#training model prediction
surface_train_pred = surface_model.predict(surface_x)

In [17]:
#accuracy scores
surface_train_acc = sk.metrics.accuracy_score(surface_y, surface_train_pred)

In [18]:
print("Train Accuracy: %.2f%%" % (surface_train_acc * 100.0))

Train Accuracy: 99.13%
